In [1]:
import random

import pandas as pd
import numpy as np
import math
from scipy import stats as scistats

import json
import array as arr
from datetime import datetime, date

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

In [2]:
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
df_stat_data = pd.read_csv('data/df_stat_data_10_25.csv')
df_fight_data = pd.read_csv('data/df_fight_data_10_25.csv',
                            parse_dates=['date'], date_parser=dateparse)

with open('data/fighter_hist.json') as json_file:
    fighter_dict = json.load(json_file)
    
with open('data/fighter_name_to_id.json') as json_file:
    name_dict = json.load(json_file)

elo b1:

    - First we calculate an elo based on winning and losing (win_loss_elo),
          very similar to chess
      
    - for each fight stat type (sig str, clinch, td, etc) we calculate
          an additional elo (ex. clinch_elo)
      
    - clinch_elo increases based on the win_loss_elo of the opponent fighter
         -being able to outland an opponent in the clinch is rewarded 
          based on the opponent's general win_loss_record.

Results: same as older methods

elo c1:
- we include regular win loss elo from older methods
- sig str = head + body + leg = distance + clinch + ground
- keep track of sig str elo
- for each fight
    - calculate league head-body-leg ratio, distance-clinch-ground ratio
        - avg and stdev
    - if someone lands more sig strikes:
        - update their sig str elo
        - update head, body, leg, distance, clinch, ground elos based on standardization with league avg
    - if someone lands less sig strikes:
        - update their sig str elo
        - update head, body, leg, distance, clinch, ground elos based on standardization with league avg

elo e1
- calculate sig str elo normally
- additionally keep track of mean, sd percent distributions of head/body/leg, distance/clinch/ground
- calculate standardized fighter distributions based on mean, sd

In [3]:
example_df = pd.DataFrame( \
              {'elo_version': ['a1', 'a1', 'a2', 'b1', 'b1', 'b1'],
               'fighter_id': ['324', '21', '154', '3213', '43', '1213'],
               'previous_card_date': ['2021-10-08', '2021-09-08', '2021-08-08',
                                 '2021-08-08', '2021-08-08', '2021-07-08'],
               'previous_fight_id': ['664', '654', '633', '633', '633', '632'],
               'elo_type': ['win_loss', 'win_loss', 'win_loss', 'sig_str', 'sig_str', 'distance'],
               'elo_value': [100, 150, 150, 400, 430, 445] ,
               'opp_prev_value': [20, 20, 20 , 20, 20, 20], 
               
               })

example_df

,elo_version,fighter_id,previous_card_date,previous_fight_id,elo_type,elo_value,opp_prev_value
0,a1,324,2021-10-08,664,win_loss,100,20
1,a1,21,2021-09-08,654,win_loss,150,20
2,a2,154,2021-08-08,633,win_loss,150,20
3,b1,3213,2021-08-08,633,sig_str,400,20
4,b1,43,2021-08-08,633,sig_str,430,20
5,b1,1213,2021-07-08,632,distance,445,20


In [4]:
df_stat_data[(df_stat_data.Stat == 'sig str') &
             (df_stat_data.Round == 0)].head()

,Fighter,Landed,Out Of,Round,Seconds,Stat,fight_id
80,0,41,80,0,628,sig str,5291
81,1,43,65,0,628,sig str,5291
132,0,4,7,0,154,sig str,5290
133,1,13,16,0,154,sig str,5290
184,0,1,2,0,44,sig str,5289


In [5]:
example_df.sort_values(by='previous_card_date', ascending=True)

,elo_version,fighter_id,previous_card_date,previous_fight_id,elo_type,elo_value,opp_prev_value
5,b1,1213,2021-07-08,632,distance,445,20
2,a2,154,2021-08-08,633,win_loss,150,20
3,b1,3213,2021-08-08,633,sig_str,400,20
4,b1,43,2021-08-08,633,sig_str,430,20
1,a1,21,2021-09-08,654,win_loss,150,20
0,a1,324,2021-10-08,664,win_loss,100,20


In [6]:
df_stat_data.head()

,Fighter,Landed,Out Of,Round,Seconds,Stat,fight_id
0,0,0,-1,1,300,kd,5291
1,1,0,-1,1,300,kd,5291
2,0,15,29,1,300,sig str,5291
3,1,13,18,1,300,sig str,5291
4,0,15,29,1,300,total str,5291


In [4]:
def gen_new_mean_sd(entry, new_val):
    mean = entry['mean']
    cnt = entry['cnt']
    sd = entry['sd']

    if (cnt == 0):
        new_mean = new_val
        new_sd = 0

    else:
        new_mean = (mean * cnt + new_val) / (cnt + 1)

        new_sd_numer = ((cnt) * (sd ** 2)) + \
            ((new_val - mean) * (new_val - new_mean))
        
        new_sd = math.sqrt(new_sd_numer / (cnt + 1))
    

    return {'mean': new_mean, 'sd': new_sd, 'cnt': cnt + 1}

In [5]:
def gen_fighter_elo_increments(fighter_0_elo, fighter_1_elo, winner, divby=400, k_val=40):
    
    prob_fighter_0 = 1.0 / (1 + 1.0 * math.pow(10, 1.0 *
                                               (fighter_1_elo - fighter_0_elo) /
                                               divby))
    prob_fighter_1 = 1 - prob_fighter_0
    
    if winner == 0:
        return k_val * (1 - prob_fighter_0), \
               k_val * (0 - prob_fighter_1) 
    
    return k_val * (0 - prob_fighter_0), \
           k_val * (1 - prob_fighter_1)

In [6]:
def melt_fight_data(df_fight_data):
    df_fight_data = df_fight_data.copy()
    df_fight_data = df_fight_data[['fighter_0', 'fighter_1', 'winner', 'date', 'fight_id']]
    returned = pd.melt(df_fight_data, id_vars=['winner', 'date', 'fight_id'])
    returned.rename(columns={'variable': 'fighter', 'value': 'fighter_id'},
        inplace=True)
    returned['fighter'] = returned['fighter']\
        .apply(lambda x: 0 if x == 'fighter_0' else 1)

    to_merge = df_fight_data[['fight_id', 'fighter_0', 'fighter_1']]
    returned = returned.merge(to_merge, on='fight_id')
    returned['opponent_id'] = returned\
        .apply(lambda x: x['fighter_1'] if x['fighter'] == 0 else x['fighter_0'],
            axis=1)

    return returned.drop(columns=['fighter_0', 'fighter_1'])
melt_fight_data(df_fight_data)\
    .sort_values(by=['fight_id', 'fighter'], ascending=True).head()

,winner,date,fight_id,fighter,fighter_id,opponent_id
0,0,1998-10-16,0,0,1899,1900
1,0,1998-10-16,0,1,1900,1899
2,0,1998-10-16,1,0,1898,1601
3,0,1998-10-16,1,1,1601,1898
4,0,1998-10-16,2,0,1813,1819


In [10]:
df_stat_data.head()

,Fighter,Landed,Out Of,Round,Seconds,Stat,fight_id
0,0,0,-1,1,300,kd,5291
1,1,0,-1,1,300,kd,5291
2,0,15,29,1,300,sig str,5291
3,1,13,18,1,300,sig str,5291
4,0,15,29,1,300,total str,5291


In [12]:
def gen_elo_e1(df_fight_data, df_stat_data, fighter_dict, name_dict, divby=400, k_val=40):

    df_fights = melt_fight_data(df_fight_data)\
        .sort_values(by=['fight_id', 'fighter'], ascending=True)

    list_returned = []
    set_fighters_observed = set()
    ranges = ('distance', 'clinch', 'ground')
    locations = ('head', 'body', 'leg')
    stat_types = ('sig str',) + ranges + locations

    stat_type_mean_sd_counts = {s: 
        {'mean': -1, 'sd': -1, 'cnt': 0} for s in locations + ranges}


    # Filter to remove round-by-round stat data
    df_stats = df_stat_data[(df_stat_data.Round == 0) &
            df_stat_data.Stat.isin(stat_types)]\
        [['Fighter', 'Landed', 'Stat', 'Seconds', 'fight_id']] \
        .set_index('fight_id')

    current_elo_values = {fighter_id: {'sig str': 100, 'win_loss': 100}
         for fighter_id in fighter_dict.keys()}

    current_strike_dstrbs = {fighter_id: {k: 0 for k in stat_types}
        for fighter_id in fighter_dict.keys()}
    

    # handle issue of figher_elo update affecting the calculation of opponent's elo
    temp_elo_store = {}

    #iterate through fight_ids
    for fight_entry in df_fights.to_dict('records'):
        
        fight_stats = df_stats.loc[fight_entry['fight_id']]

        # get fighter ids
        fighter_id = str(fight_entry['fighter_id'])
        opp_id = str(fight_entry['opponent_id'])

        # determine dictionaries to be appended to returned

        to_append = {'fighter_id': fighter_id,
             'card_date': fight_entry['date'],
             'fight_id': fight_entry['fight_id'], 
             'fighter' : fight_entry['fighter'],
             'winner': fight_entry['winner']}

        to_append.update({stat_type: current_elo_values[fighter_id][stat_type]
            for stat_type in ('sig str', 'win_loss')})


        def get_current_stat_sd(stat_type):
            mean_sd_entry = stat_type_mean_sd_counts[stat_type]
            fighter_stat_types = current_strike_dstrbs[fighter_id]

            stat_type_perc = fighter_stat_types[stat_type] / fighter_stat_types['sig str']\
                if fighter_stat_types['sig str'] != 0 else 1/3

            z_val = (stat_type_perc - mean_sd_entry['mean']) / mean_sd_entry['sd']\
                if (mean_sd_entry['sd'] != 0 ) and (mean_sd_entry['cnt'] != 0 ) else 0

            return  scistats.norm.cdf(z_val) * current_elo_values[fighter_id]['sig str']

        # for each range and location striking stat, add a column
        # containing percentage of total sig strikes
        to_append.update({
            
            stat_type: get_current_stat_sd(stat_type)

            # iterate through ranges and locations
            for stat_type in (ranges + locations)
        })

        list_returned.append(to_append)

        # Now we focus on updating elo values appropriately

        fighter_wl_elo = current_elo_values[fighter_id]['win_loss']
        opp_wl_elo = current_elo_values[opp_id]['win_loss']

        fighter_ss_elo = current_elo_values[fighter_id]['sig str']
        opp_ss_elo = current_elo_values[opp_id]['sig str']

        fighter_observed = fighter_id in set_fighters_observed
        opp_observed = opp_id in set_fighters_observed


        
        if fight_entry['fighter'] == 1:
            set_fighters_observed.add(fighter_id)
            set_fighters_observed.add(opp_id)

        obs_info = 'normal'
        
        if (not fighter_observed) and opp_observed:
            fighter_wl_elo = opp_wl_elo
            fighter_ss_elo = opp_ss_elo
            obs_info = 'fighter_new'

        if fighter_observed and (not opp_observed):
            opp_wl_elo = fighter_wl_elo
            opp_ss_elo = fighter_ss_elo
            obs_info = 'fighter_new'
            
        if (not fighter_observed) and (not opp_observed):
            fighter_wl_elo = 100
            fighter_ss_elo = 100
            obs_info = 'both_new'


        def get_landed(df_stats, stat_type, fighter):
            single_row = df_stats[(df_stats.Fighter == fighter) & 
                (df_stats.Stat == stat_type)]

            if len(single_row) != 1:
                raise Exception('invalid params')

            return single_row.iloc[0]['Landed']

       
        fighter_bin = fight_entry['fighter']
        opp_bin = 1 - fighter_bin

        elo_winner = int(fight_entry['winner'] != fighter_bin)
        ss_landed_compare = get_landed(fight_stats, 'sig str', fighter_bin) - \
            get_landed(fight_stats, 'sig str', opp_bin)
        
        fighter_wl_inc, _ = \
            gen_fighter_elo_increments(fighter_wl_elo, opp_wl_elo, elo_winner)
        
        fighter_ss_inc = \
            gen_fighter_elo_increments(fighter_ss_elo, opp_ss_elo,
                (int(ss_landed_compare < 0)))[0] \
                    if ss_landed_compare != 0 else 0
        
        new_fighter_wl_elo = fighter_wl_elo + fighter_wl_inc
        new_fighter_ss_elo = fighter_ss_elo + fighter_ss_inc

        ss_landed = get_landed(fight_stats, 'sig str', fighter_bin)

        for stat_type in stat_types:

            num_landed = get_landed(fight_stats, stat_type, fighter_bin)
            perc_landed = num_landed / ss_landed if ss_landed != 0 else 1/3

            current_strike_dstrbs[fighter_id][stat_type] = \
                current_strike_dstrbs[fighter_id][stat_type] + num_landed

            if stat_type != 'sig str':   
                new_entry = gen_new_mean_sd(stat_type_mean_sd_counts[stat_type], perc_landed)
                
                stat_type_mean_sd_counts[stat_type] = new_entry
            
        elo_updates = {'sig str': new_fighter_ss_elo,
                       'win_loss': new_fighter_wl_elo}
        if fighter_bin == 0:
            temp_elo_store.update(elo_updates)
        else:
            current_elo_values[opp_id] = temp_elo_store.copy()
            temp_elo_store.clear()
            current_elo_values[fighter_id] = elo_updates.copy()

    return pd.DataFrame.from_records(list_returned), current_elo_values, stat_type_mean_sd_counts, current_strike_dstrbs
        

In [53]:
def gen_prediction_e1(model, fighter0_id, fighter1_id, current_elo_values, 
    stat_type_mean_sd_counts, current_strike_dstrbs, stat_types, win_loss=True, sig_str=False, divby=400):

    to_feed = []

    for stat_type in stat_types:

        if stat_type in ('win_loss', 'sig str'):
            elos = [current_elo_values[fighter0_id][stat_type],
                current_elo_values[fighter1_id][stat_type]]

        elif stat_type in ('head', 'body', 'leg', 'distance', 'clinch', 'ground'):
            elos = []
            mean_sd_entry = stat_type_mean_sd_counts[stat_type]

            for fighter_id in (fighter0_id, fighter1_id):
                fighter_stat_types = current_strike_dstrbs[fighter_id]

                stat_type_perc = fighter_stat_types[stat_type] / fighter_stat_types['sig str']\
                    if fighter_stat_types['sig str'] != 0 else 1/3

                z_val = (stat_type_perc - mean_sd_entry['mean']) / mean_sd_entry['sd']\
                    if (mean_sd_entry['sd'] != 0 ) and (mean_sd_entry['cnt'] != 0 ) else 0

                elos.append(scistats.norm.cdf(z_val) * current_elo_values[fighter_id]['sig str'])

        to_feed.append(1.0 / (1 + 1.0 * math.pow(10, (elos[1] - elos[0]) / divby)))

    return model.predict([to_feed])

gen_prediction_e1(mod, '422', '212', current_elo_values, 
    stat_type_mean_sd_counts, current_strike_dstrbs, ['head', 'body', 'leg', 'distance', 'clinch', 'ground', 'win_loss'], win_loss=True, sig_str=False, divby=400)

array([1.])

In [69]:
with open('data/current_elo_values.json') as json_file:
    current_elo_values_load = json.load(json_file)

with open('data/stat_type_mean_sd_counts.json') as json_file:
    stat_type_mean_sd_counts_load = json.load(json_file)

with open('data/current_strike_dstrbs.json') as json_file:
    current_strike_dstrbs_load = json.load(json_file)

gen_prediction_e1(mod, '422', '212', current_elo_values_load, 
    stat_type_mean_sd_counts_load, current_strike_dstrbs_load, 
    ['head', 'body', 'leg', 'distance', 'clinch', 'ground', 'win_loss'], win_loss=True, sig_str=False, divby=400)

array([1.])

In [52]:
name_dict['Max Holloway']

212

In [ ]:
# e1_results = gen_elo_e1(df_fight_data[(df_fight_data.fighter_0 == 118) | 
#     (df_fight_data.fighter_1 == 118)],
#      df_stat_data, fighter_dict, name_dict, divby=400, k_val=40)

df_results, current_elo_values, stat_type_mean_sd_counts, current_strike_dstrbs = \
    gen_elo_e1(df_fight_data,
        df_stat_data, fighter_dict, name_dict, divby=400, k_val=40)

In [66]:
type(int(current_strike_dstrbs['0']['distance']))

int

In [67]:
def fix_int32(current_strike_dstrbs):
    return {fid: {st: int(x) for st, x in fidval.items()} for fid, fidval in current_strike_dstrbs.items()}

In [68]:
with open("data/current_elo_values.json", "w") as f:
    json.dump(current_elo_values, f)

with open("data/stat_type_mean_sd_counts.json", "w") as f:
    json.dump(stat_type_mean_sd_counts, f)

with open("data/current_strike_dstrbs.json", "w") as f:
    json.dump(fix_int32(current_strike_dstrbs), f)


In [36]:
df_results.sort_values(by='head')

,body,card_date,clinch,distance,fight_id,fighter,fighter_id,ground,head,leg,sig str,win_loss,winner
9292,-18.277154,2018-12-01,-17.293796,-3.405553e+01,4646,0,567,-1.785809e+01,-3.827534e+01,-11.031861,-48.355462,51.779747,0
8695,-12.415259,2018-04-21,-12.700274,-2.388205e+01,4347,1,567,-1.327262e+01,-2.799941e+01,-7.676711,-34.684672,69.263557,0
10226,-7.262265,2019-10-18,-13.639698,-1.056509e+01,5113,0,333,-1.787379e+01,-2.413094e+01,-5.597127,-27.775357,156.321049,0
8156,-6.980816,2017-09-22,-10.321969,-1.763859e+01,4078,0,567,-1.069697e+01,-2.294988e+01,-5.728918,-26.681929,41.277556,0
9667,-8.044291,2019-04-20,-8.446314,-1.655235e+01,4833,1,567,-8.116425e+00,-1.826451e+01,-5.934554,-23.166079,75.726313,0
3667,-7.148166,2012-08-04,-9.923033,-1.437373e+01,1833,1,1368,-8.443961e+00,-1.744416e+01,-6.369476,-22.245183,49.836950,0
7276,-3.883099,2016-10-01,-5.736456,-1.033649e+01,3638,0,567,-5.763776e+00,-1.298917e+01,-3.360597,-15.124510,58.711271,1
8656,-3.811866,2018-04-07,-6.937129,-5.624498e+00,4328,0,333,-9.833436e+00,-1.291873e+01,-3.016394,-14.857355,181.708300,1
7718,-6.730582,2017-03-18,-6.949244,-1.195586e+01,3859,0,849,-6.381956e+00,-9.190345e+00,-9.976860,-17.474178,68.129600,1
3738,-4.292626,2012-09-29,-1.540927,-7.912167e+00,1869,0,1318,-1.733833e+00,-6.557155e+00,-1.609128,-8.597385,111.845915,1


In [8]:
def gen_elo_c2(df_fight_data, df_stat_data, fighter_dict, name_dict, divby=400, k_val=40):
    
    df_fights = melt_fight_data(df_fight_data)\
        .sort_values(by=['fight_id', 'fighter'], ascending=True)


    list_win_loss = []
    set_fighters_observed = set()

    ranges = ('distance', 'clinch', 'ground')
    locations = ('head', 'body', 'leg')
    stat_types = ('sig str',) + ranges + locations

    range_vals = {r: {'mean': -1, 'sd': -1, 'cnt': 0} for r in ranges}
    loc_vals = {l: {'mean': -1, 'sd': -1, 'cnt': 0} for l in locations}
    
    # Filter to remove round-by-round stat data
    df_stats = df_stat_data[(df_stat_data.Round == 0) &
            df_stat_data.Stat.isin(stat_types)]\
        [['Fighter', 'Landed', 'Stat', 'Seconds', 'fight_id']] \
        .rename(columns={'Fighter': 'fighter'})\
        .set_index('fight_id')
    
    current_elo_values = {f_id: 
                             {elo_type: 0 
                                for elo_type in stat_types + ('win_loss',)}
                                    for f_id in fighter_dict.keys()}

    set_fighters_observed = set()

    # handle issue of figher_elo update affecting the calculation of opponent's elo
    temp_elo_store = {}

    #iterate through fight_ids
    for fight_entry in df_fights.to_dict('records'):
        
        fight_stats = df_stats.loc[fight_entry['fight_id']]

        # get fighter ids
        fighter_id = str(fight_entry['fighter_id'])
        opp_id = str(fight_entry['opponent_id'])

        # determine dictionaries to be appended to returned

        to_append = {'fighter_id': fighter_id,
             'card_date': fight_entry['date'],
             'fight_id': fight_entry['fight_id'], 
             'fighter' : fight_entry['fighter'],
             'winner': fight_entry['winner']}

        to_append.update({stat_type: current_elo_values[fighter_id][stat_type]
            for stat_type in stat_types + ('win_loss',)})

        # append to returned
        list_win_loss.append(to_append)
        
        # Now we focus on updating elo values appropriately

        fighter_wl_elo = current_elo_values[fighter_id]['win_loss']
        opp_wl_elo = current_elo_values[opp_id]['win_loss']

        fighter_ss_elo = current_elo_values[fighter_id]['sig str']
        opp_ss_elo = current_elo_values[opp_id]['sig str']

        fighter_wl_inc, _ = \
            gen_fighter_elo_increments(fighter_wl_elo, opp_wl_elo,
                 int(fight_entry['winner'] != fight_entry['fighter']))
        
        fighter_ss_inc, _ = \
            gen_fighter_elo_increments(fighter_ss_elo, opp_ss_elo,
                 int(fight_entry['winner'] != fight_entry['fighter']))
                  
        # take the increment from sig str elos, and divide it based on r_data
        #  get the stat values from df_stats

        fighter_observed = fighter_id in set_fighters_observed
        opp_observed = opp_id in set_fighters_observed
        
        if fight_entry['fighter'] == 1:
            set_fighters_observed.add(fighter_id)
            set_fighters_observed.add(opp_id)

        obs_info = 'normal'
        
        if (not fighter_observed) and opp_observed:
            fighter_wl_elo = opp_wl_elo
            fighter_ss_elo = opp_ss_elo
            obs_info = 'current_new'
            
        if (not fighter_observed) and (not opp_observed):
            fighter_wl_elo = 0
            fighter_ss_elo = 0
            obs_info = 'both_new'

        def get_stat_landed(stat_name):
            return list(fight_stats[(fight_stats.fighter ==  0) &
            (fight_stats.Stat == stat_name)]['Landed'])[0]

        stat_elo_update = {'win_loss': fighter_wl_elo + fighter_wl_inc,
                'sig str': fighter_ss_elo + fighter_ss_inc}

        for piece_stats, piece_vals in ((ranges, range_vals), (locations, loc_vals)):

            range_landed = {r: get_stat_landed(r) for r in piece_stats}
            sum_landed = sum(list(range_landed.values()))
            range_landed_perc = {r: val/(sum_landed if sum_landed != 0 else 1)
                 for r, val in range_landed.items()}
            range_standardized = {}
            

            for stat_name, perc_landed in range_landed_perc.items():
                new_stat_entry = gen_new_mean_sd(piece_vals[stat_name], perc_landed)
                piece_vals[stat_name] = new_stat_entry
                range_standardized[stat_name] = (perc_landed - new_stat_entry['mean']) / \
                    (new_stat_entry['sd'] if new_stat_entry['sd'] != 0 else 1)

            range_standardized = {k: scistats.norm.cdf(v) * 2 / 3
                for k, v in range_standardized.items()}

            for stat_name, inc_perc in range_standardized.items():

                initial_elo = current_elo_values[fighter_id][stat_name] if obs_info == 'normal' \
                    else current_elo_values[opp_id]['sig str'] / 3 if obs_info == 'current_new' \
                        else 0

                stat_elo_update[stat_name] = initial_elo + \
                    (fighter_ss_inc * inc_perc if fight_entry['winner'] == fight_entry['fighter']
                        else fighter_ss_inc * (2/3 - inc_perc))

        if fight_entry['fighter'] == 0:
            temp_elo_store.update(stat_elo_update)
        else:
            current_elo_values[opp_id] = temp_elo_store.copy()
            temp_elo_store.clear()
            current_elo_values[fighter_id] = stat_elo_update.copy()
    
    df_returned = pd.DataFrame.from_records(list_win_loss)
    return df_returned

        
df_c2 = gen_elo_c2(df_fight_data, df_stat_data, fighter_dict, name_dict, divby=400, k_val=40)

In [149]:
def gen_elo_d1(df_fight_data, df_stat_data, fighter_dict, name_dict, divby=400, k_val=40):
    
    df_fights = melt_fight_data(df_fight_data)\
        .sort_values(by=['fight_id', 'fighter'], ascending=True)


    list_win_loss = []
    set_fighters_observed = set()
    
    
    # Filter to remove round-by-round stat data
    df_stats = df_stat_data[(df_stat_data.Round == 0) &
            df_stat_data.Stat.isin(stat_types)]\
        [['Fighter', 'Landed', 'Stat', 'Seconds', 'fight_id']] \
        .rename(columns={'Fighter': 'fighter'})\
        .set_index('fight_id')

    stat_vals = {s: {'mean': -1, 'sd': -1, 'cnt': 0} for s in stat_types}
    
    current_elo_values = {f_id: 
                             {elo_type: 0 
                                for elo_type in stat_types + ('win_loss',)}
                                    for f_id in fighter_dict.keys()}

    set_fighters_observed = set()

    # handle issue of figher_elo update affecting the calculation of opponent's elo
    temp_elo_store = {}

    #iterate through fight_ids
    for fight_entry in df_fights.to_dict('records'):
        
        fight_stats = df_stats.loc[fight_entry['fight_id']]

        # get fighter ids
        fighter_id = str(fight_entry['fighter_id'])
        opp_id = str(fight_entry['opponent_id'])

        # determine dictionaries to be appended to returned

        to_append = {'fighter_id': fighter_id,
             'card_date': fight_entry['date'],
             'fight_id': fight_entry['fight_id'], 
             'fighter' : fight_entry['fighter'],
             'winner': fight_entry['winner']}

        to_append.update({stat_type: current_elo_values[fighter_id][stat_type]
            for stat_type in stat_types + ('win_loss',)})

        # append to returned
        list_win_loss.append(to_append)
        
        # Now we focus on updating elo values appropriately

        fighter_wl_elo = current_elo_values[fighter_id]['win_loss']
        opp_wl_elo = current_elo_values[opp_id]['win_loss']

        fighter_ss_elo = current_elo_values[fighter_id]['sig str']
        opp_ss_elo = current_elo_values[opp_id]['sig str']

        fighter_wl_inc, _ = \
            gen_fighter_elo_increments(fighter_wl_elo, opp_wl_elo,
                 int(fight_entry['winner'] != fight_entry['fighter']))
        
        fighter_ss_inc, _ = \
            gen_fighter_elo_increments(fighter_ss_elo, opp_ss_elo,
                 int(fight_entry['winner'] != fight_entry['fighter']))
                  
        # take the increment from sig str elos, and divide it based on r_data
        #  get the stat values from df_stats

        fighter_observed = fighter_id in set_fighters_observed
        opp_observed = opp_id in set_fighters_observed
        
        if fight_entry['fighter'] == 1:
            set_fighters_observed.add(fighter_id)
            set_fighters_observed.add(opp_id)

        obs_info = 'normal'
        
        if (not fighter_observed) and opp_observed:
            fighter_wl_elo = opp_wl_elo
            fighter_ss_elo = opp_ss_elo
            obs_info = 'current_new'
            
        if (not fighter_observed) and (not opp_observed):
            fighter_wl_elo = 0
            fighter_ss_elo = 0
            obs_info = 'both_new'

        def get_stat_landed(stat_name):
            return list(fight_stats[(fight_stats.fighter ==  0) &
            (fight_stats.Stat == stat_name)]['Landed'])[0]

        stat_elo_update = {'win_loss': fighter_wl_elo + fighter_wl_inc,
                'sig str': fighter_ss_elo + fighter_ss_inc}

        for piece_stats, piece_vals in ((ranges, range_vals), (locations, loc_vals)):

            range_landed = {r: get_stat_landed(r) for r in piece_stats}
            sum_landed = sum(list(range_landed.values()))
            range_landed_perc = {r: val/(sum_landed if sum_landed != 0 else 1)
                 for r, val in range_landed.items()}
            range_standardized = {}
            

            for stat_name, perc_landed in range_landed_perc.items():
                new_stat_entry = gen_new_mean_sd(piece_vals[stat_name], perc_landed)
                piece_vals[stat_name] = new_stat_entry
                range_standardized[stat_name] = (perc_landed - new_stat_entry['mean']) / \
                    (new_stat_entry['sd'] if new_stat_entry['sd'] != 0 else 1)

            range_standardized = {k: scistats.norm.cdf(v) * 2 / 3
                for k, v in range_standardized.items()}

            for stat_name, inc_perc in range_standardized.items():

                initial_elo = current_elo_values[fighter_id][stat_name] if obs_info == 'normal' \
                    else current_elo_values[opp_id]['sig str'] / 3 if obs_info == 'current_new' \
                        else 0

                stat_elo_update[stat_name] = initial_elo + \
                    (fighter_ss_inc * inc_perc if fight_entry['winner'] == fight_entry['fighter']
                        else fighter_ss_inc * (2/3 - inc_perc))

        if fight_entry['fighter'] == 0:
            temp_elo_store.update(stat_elo_update)
        else:
            current_elo_values[opp_id] = temp_elo_store.copy()
            temp_elo_store.clear()
            current_elo_values[fighter_id] = stat_elo_update.copy()
    
    df_returned = pd.DataFrame.from_records(list_win_loss)
    return df_returned

        
df_c2 = gen_elo_c2(df_fight_data, df_stat_data, fighter_dict, name_dict, divby=400, k_val=40)

In [7]:
def gen_elo_c1(df_fight_data, df_stat_data, fighter_dict, name_dict, divby=400, k_val=40):
    

    stat_types = ('sig str') + ranges + locations
    current_elo_values = {f_id: 
                             {elo_type: 0 
                                for elo_type in stat_types + ['win_loss']}
                                    for f_id in fighter_dict.keys()}

    list_win_loss = []
    set_fighters_observed = set()

    # sort fights by date
    df_fights = df_fight_data \
        .sort_values(by='fight_id', ascending=True) \
        .set_index('fight_id')
    
    # Filter to remove round-by-round stat data
    df_stats = df_stat_data[(df_stat_data.Round == 0) &
            df_stat_data.Stat.isin(stat_types)]\
        [['Fighter', 'Landed', 'Stat', 'Seconds', 'fight_id']] \
        .rename(columns={'Fighter': 'fighter'})\
        .set_index('fight_id')

    ranges = ('distance', 'clinch', 'ground')
    locations = ('head', 'body', 'leg')

    range_vals = {r: {'mean': -1, 'sd': -1, 'cnt': 0} for r in ranges}
    loc_vals = {l: {'mean': -1, 'sd': -1, 'cnt': 0} for l in locations}

    set_fighters_observed = set()
    
    #iterate through fight_ids
    for fight_id in df_fights.index:
        
        #
        fight_stats = df_stats.loc[fight_id]
        
        # dictionary containing data about the fight
        fight_entry = dict(df_fight_data.loc[fight_id])

        # get fighter ids
        fighter_0_id = str(fight_entry['fighter_0'])
        fighter_1_id = str(fight_entry['fighter_1'])

        # determine dictionaries to be appended to returned

        to_append_0 = {'fighter_id': fighter_0_id,
             'card_date': fight_entry['date'],
             'fight_id': fight_entry['fight_id'], 
             'fighter' : 0,
             'winner': fight_entry['winner']}

        to_append_0.update({current_elo_values[fighter_0_id][stat_type]
            for stat_type in stat_types})

        to_append_1 = {'fighter_id': fighter_1_id,
             'card_date': fight_entry['date'],
             'fight_id': fight_entry['fight_id'], 
             'fighter' : 1,
             'winner': fight_entry['winner']}

        to_append_1.update({current_elo_values[fighter_1_id][stat_type]
            for stat_type in stat_types})

        # append to returned
        list_win_loss.append(to_append_0)
        list_win_loss.append(to_append_1)
        
        # Now we focus on updating elo values appropriately

        fighter_0_wl_elo = current_elo_values[fighter_0_id]['win_loss']
        fighter_1_wl_elo = current_elo_values[fighter_1_id]['win_loss']

        fighter_0_ss_elo = current_elo_values[fighter_0_id]['sig str']
        fighter_1_ss_elo = current_elo_values[fighter_1_id]['sig str']

        fighter_0_ss_inc, fighter_1_ss_inc = \
            gen_fighter_elo_increments(fighter_0_ss_elo, fighter_1_ss_elo,
                 fight_entry['winner'])
                  
        # take the increment from sig str elos, and divide it based on r_data
        #  get the stat values from df_stats

        def get_stat_landed(stat_name):
            return list(fight_stats[(fight_stats.Fighter ==  0) &
            (fight_stats.Stat == stat_name)])[0]

        range_landed = {r: get_stat_landed(r) for r in ranges}
        sum_landed = sum(list(range_landed.values()))
        range_landed_perc = {r: val/sum_landed for r, val in range_landed.items()}
        range_standardized = {}

        for stat_name, perc_landed in range_landed_perc.items():
            new_stat_entry = gen_new_mean_sd(range_vals[stat_name], perc_landed)
            range_vals[stat_name] = new_stat_entry
            range_standardized[stat_name] = (perc_landed - new_stat_entry['mean']) / \
                new_stat_entry['sd']
            
  

In [9]:
def gen_elo_b1(df_fight_data, df_stat_data, fighter_dict, name_dict, divby=400, k_val=40):
    
    # First we create a dictionary to store the current values of the elos
    # its current structure is as follows:

    #     {fighter_id: 
    #         {elo_type: elo_value} (contains all zeroes to begin)
    #     }

    # list of different stat types
    stat_types = list(df_stat_data.Stat.unique())

    current_elo_values = {f_id: 
                             {elo_type: 0 
                                for elo_type in stat_types + ['win_loss']}
                                    for f_id in fighter_dict.keys()}
    
    # Next we create a dataframe containing only only the win_loss_elo data
    # It has the same structure as the example df except for it doesn't have the elotype column,
    # and the elo value column is replaced with the column name of the stat type.
    
    # empty list to begin
    list_win_loss = []
    
    set_fighters_observed = set()
    
    # sort fights by date
    df_fights = df_fight_data \
        .sort_values(by='fight_id', ascending=True) \
        .set_index('fight_id')
    
    # Filter to remove round-by-round stat data
    df_stats = df_stat_data[df_stat_data.Round == 0][['Fighter', 'Landed', 'Stat', 'fight_id']] \
        .rename(columns={'Fighter': 'fighter'})\
        .set_index('fight_id')
    
    #iterate through fight_ids
    for fight_id in df_fights.index:
        
        #
        fight_stats = df_stats.loc[fight_id]
        
        # dictionary containing data about the fight
        fight_entry = dict(df_fight_data.loc[fight_id])
        
        # get fighter ids
        fighter_0_id = str(fight_entry['fighter_0'])
        fighter_1_id = str(fight_entry['fighter_1'])

        # get current fighter wl elos
        fighter_0_wl_elo = current_elo_values[fighter_0_id]['win_loss']
        fighter_1_wl_elo = current_elo_values[fighter_1_id]['win_loss']
        
        fighter_0_observed = fighter_0_id in set_fighters_observed
        fighter_1_observed = fighter_1_id in set_fighters_observed
        
        set_fighters_observed.add(fighter_0_id)
        set_fighters_observed.add(fighter_1_id)
        
        if (not fighter_0_observed) and fighter_1_observed:
            fighter_0_wl_elo = fighter_1_wl_elo
            
        if fighter_0_observed and (not fighter_1_observed):
            fighter_1_wl_elo = fighter_0_wl_elo
        
        # get new win_loss elo values
        fighter_0_wl_inc, fighter_1_wl_inc = \
            gen_fighter_elo_increments(fighter_0_wl_elo, fighter_1_wl_elo, fight_entry['winner'])
        
        fighter_0_wl_elo_new = fighter_0_wl_inc + fighter_0_wl_elo
        fighter_1_wl_elo_new = fighter_1_wl_inc + fighter_1_wl_elo
        
        # get dictionary of other new elo values
        dict_elos_0, dict_elos_1 = {'old': {}, 'new': {}}, {'old': {}, 'new': {}}
        for stat_type in stat_types:
            
            fighter_0_stat_elo = current_elo_values[fighter_0_id][stat_type]
            fighter_1_stat_elo = current_elo_values[fighter_1_id][stat_type]
            
            dict_elos_0['old'][stat_type] = fighter_0_stat_elo
            dict_elos_1['old'][stat_type] = fighter_1_stat_elo  
            
            relevant_stats = fight_stats[(fight_stats['Stat'] == stat_type)]
            fighter_0_landed = list(relevant_stats[relevant_stats['fighter'] == 0]['Landed'])[0]
            fighter_1_landed = list(relevant_stats[relevant_stats['fighter'] == 1]['Landed'])[0]
            
            if ((fight_entry['winner'] == 0) and (fighter_0_landed > fighter_1_landed)) or \
                ((fight_entry['winner'] == 1) and (fighter_0_landed < fighter_1_landed)):
                
                fighter_0_stat_elo = fighter_0_stat_elo + fighter_0_wl_inc
                fighter_1_stat_elo = fighter_1_stat_elo + fighter_1_wl_inc

                
            dict_elos_0['new'][stat_type] = fighter_0_stat_elo
            dict_elos_1['new'][stat_type] = fighter_1_stat_elo     
        
        to_append_0 = {'fighter_id': fighter_0_id,
             'card_date': fight_entry['date'],
             'fight_id': fight_entry['fight_id'], 
             'fighter' : 0,
             'winner': fight_entry['winner'],
             'win_loss': current_elo_values[fighter_0_id]['win_loss']}
        
        to_append_0.update(dict_elos_0['old'])
        
        to_append_1 = {'fighter_id': fighter_1_id,
             'card_date': fight_entry['date'],
             'fight_id': fight_entry['fight_id'], 
             'fighter': 1,
             'winner': fight_entry['winner'],
             'win_loss': current_elo_values[fighter_1_id]['win_loss']}
    
        to_append_1.update(dict_elos_1['old'])
    
        # append relevant data to future dataframe
        list_win_loss.append(to_append_0)
        list_win_loss.append(to_append_1)
        
        # update current_elo_values
        current_elo_values[fighter_0_id]['win_loss'] = fighter_0_wl_elo_new
        current_elo_values[fighter_1_id]['win_loss'] = fighter_1_wl_elo_new
        
        for stat_type in stat_types:
            current_elo_values[fighter_0_id][stat_type] = dict_elos_0['new'][stat_type]
            current_elo_values[fighter_1_id][stat_type] = dict_elos_1['new'][stat_type]
                       
    
    # create a dataframe from list of dictionaries list_win_loss
    # this dataframe will be eventually returned
    df_returned = pd.DataFrame.from_records(list_win_loss)
    return df_returned




In [ ]:
class FightPredictor:

    
    def __init__(self, df_elo, df_fight_data, fighter_dict, current_elo_values, 
        stat_type_mean_sd_counts, current_strike_dstrbs, columns):

        """Takes as input:
            df_elo containing historical data for all fights and all fighter
         scores and metrics at the time of the fight (df_elo)"""
        
        self.model = self.get_model(df_elo, df_fight_data, columns)
    

    def get_model(self, df_elo, df_fight_data, columns):
        df_elo = df_elo.copy()
        df_elo = df_elo[['fight_id', 'fighter', 'fighter_id'] + columns]
        df_fight_data = df_fight_data.copy()
        df_fight_data['fighter_0'] = df_fight_data['fighter_0'].astype(str)
        df_fight_data['fighter_1'] = df_fight_data['fighter_1'].astype(str)
        
        df_fight_data = df_fight_data[['fighter_0', 'fighter_1', 'fight_id', 'winner']]

        df_elo_f0 = df_elo[df_elo['fighter'] == 0]\
            .drop(columns=['fighter'])\
            .rename(columns={'fighter_id': 'fighter_0'})

        df_elo_f1 = df_elo[df_elo['fighter'] == 1]\
            .drop(columns=['fighter'])\
            .rename(columns={'fighter_id': 'fighter_1'})
        
        df_fight_data = df_fight_data.merge(df_elo_f0, on=['fight_id', 'fighter_0'], how='inner')
        df_fight_data.rename(columns={c: c + '_0' for c in columns},
            inplace=True)
        
        df_fight_data = df_fight_data.merge(df_elo_f1, on=['fight_id', 'fighter_1'], how='inner')
        df_fight_data.rename(columns={c: c + '_1' for c in columns},
            inplace=True)
            
        for col in columns:
            df_fight_data[col] = df_fight_data[col + '_1'] - df_fight_data[col + '_0']
            df_fight_data[col] = df_fight_data[col].apply(lambda x: \
                1.0 / (1 + 1.0 * math.pow(10, x / divby)))
            df_fight_data.drop(columns = [col + '_0', col + '_1'], inplace=True)

        df_fight_data = df_fight_data[columns + ['winner']]

        for i in df_fight_data.index:
            if random.randint(0, 1) == 0:
                df_fight_data.loc[i] = [1-x for x in df_fight_data.loc[i]]

        x = df_fight_data[filter(lambda x: x != 'winner', df_fight_data.columns)]
        y = df_fight_data[['winner']]
        x_train, x_test, y_train, y_test =\
            train_test_split(x, y, test_size=0.25, random_state=random_state)
        model = LogisticRegression().fit(x_train, y_train)
        accuracy = model.score(x_test, y_test)
        return model, df_fight_data, accuracy


In [56]:
def run_model(df_elo, df_fight_data, columns, divby=400, random_state=0):

    df_elo = df_elo.copy()
    df_elo = df_elo[['fight_id', 'fighter', 'fighter_id'] + columns]
    df_fight_data = df_fight_data.copy()
    df_fight_data['fighter_0'] = df_fight_data['fighter_0'].astype(str)
    df_fight_data['fighter_1'] = df_fight_data['fighter_1'].astype(str)
    
    df_fight_data = df_fight_data[['fighter_0', 'fighter_1', 'fight_id', 'winner']]

    df_elo_f0 = df_elo[df_elo['fighter'] == 0]\
        .drop(columns=['fighter'])\
        .rename(columns={'fighter_id': 'fighter_0'})

    df_elo_f1 = df_elo[df_elo['fighter'] == 1]\
        .drop(columns=['fighter'])\
        .rename(columns={'fighter_id': 'fighter_1'})
    
    df_fight_data = df_fight_data.merge(df_elo_f0, on=['fight_id', 'fighter_0'], how='inner')
    df_fight_data.rename(columns={c: c + '_0' for c in columns},
        inplace=True)
    
    df_fight_data = df_fight_data.merge(df_elo_f1, on=['fight_id', 'fighter_1'], how='inner')
    df_fight_data.rename(columns={c: c + '_1' for c in columns},
        inplace=True)
        
    for col in columns:
        df_fight_data[col] = df_fight_data[col + '_1'] - df_fight_data[col + '_0']
        df_fight_data[col] = df_fight_data[col].apply(lambda x: \
            1.0 / (1 + 1.0 * math.pow(10, x / divby)))
        df_fight_data.drop(columns = [col + '_0', col + '_1'], inplace=True)

    df_fight_data = df_fight_data[columns + ['winner']]

    for i in df_fight_data.index:
        if random.randint(0, 1) == 0:
            df_fight_data.loc[i] = [1-x for x in df_fight_data.loc[i]]

    x = df_fight_data[filter(lambda x: x != 'winner', df_fight_data.columns)]
    y = df_fight_data[['winner']]
    x_train, x_test, y_train, y_test =\
         train_test_split(x, y, test_size=0.25, random_state=random_state)
    model = LogisticRegression().fit(x_train, y_train)
    accuracy = model.score(x_test, y_test)
    return model, df_fight_data, accuracy


In [59]:

mod, test_data, accuracy = run_model(df_results, df_fight_data, ['win_loss', 'head', 'body', 'leg', 'distance', 'clinch', 'ground'])

C:\Users\augus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\augus\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [60]:
accuracy

0.5857898715041572

In [ ]:
df_results.to_csv('data/elo_results_11_30.csv')


In [86]:
mod, test = run_model(df_elo1, df_fight_data, ['sig str'])

0.581254724111867


C:\Users\augus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\augus\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [181]:
{testa.columns[i]: moda.coef_[0][i] for i in range(len(testa.columns) - 1)}

{'head': 0.09358926429035991,
 'body': -0.26853299952304466,
 'leg': -0.7724240092447986,
 'distance': -0.16628331808916783,
 'clinch': 0.11988132459690594,
 'ground': -0.6172972102264404,
 'win_loss': -2.1698236096115777}

In [23]:
mod.coef_

array([[-1.83401206, -0.17583984, -0.50560208, -1.18171317,  0.41548856,
        -1.42031812, -1.20672668]])

In [18]:
df_fight_data[df_fight_data.date > datetime(2020, 1, 1)]

,fight_name,fighter_0,fighter_1,winner,method,round_end,date,fight_id
5199,Sabina Mazo v JJ Aldrich 2020-01-18,182,183,0,Decision,3,2020-01-18,5199
5200,Aleksa Camur v Justin Ledet 2020-01-18,180,181,0,Decision,3,2020-01-18,5200
5201,Drew Dober v Nasrat Haqparast 2020-01-18,178,179,0,KO/TKO,1,2020-01-18,5201
5202,Tim Elliott v Askar Askarov 2020-01-18,176,177,1,Decision,3,2020-01-18,5202
5203,Andre Fili v Sodiq Yusuff 2020-01-18,174,175,1,Decision,3,2020-01-18,5203
5204,Roxanne Modafferi v Maycee Barber 2020-01-18,172,173,0,Decision,3,2020-01-18,5204
5205,Anthony Pettis v Diego Ferreira 2020-01-18,170,171,1,Submission,2,2020-01-18,5205
5206,Brian Kelleher v Ode Osbourne 2020-01-18,168,169,0,Submission,1,2020-01-18,5206
5207,Aleksei Oleinik v Maurice Greene 2020-01-18,166,167,0,Submission,2,2020-01-18,5207
5208,Holly Holm v Raquel Pennington 2020-01-18,164,165,0,Decision,3,2020-01-18,5208


In [57]:
testa

NameError: name 'testa' is not defined

In [16]:
stat_types = list(df_stat_data.Stat.unique())


In [ ]:

    
def gen_relevant(self, names):
        returned = {}
        v2 = self.stat_data[(self.stat_data.Stat.isin(names)) &
                           (self.stat_data.Round == 0)]
        f1_stats = v2[v2.Fighter == 0]
        f2_stats = v2[v2.Fighter == 1]
        
        for fight in self.fight_data.index:
            returned[fight] = {'fighter_0': self.fight_data['fighter_0'][fight],
                               'fighter_1': self.fight_data['fighter_1'][fight],
                               'winner': self.fight_data['winner'][fight],
                               'date': self.fight_data['date'][fight],
                               'landed_0': {name: f1_stats[f1_stats.Stat==name]['Landed'][fight] for name in names},
                               'landed_1': {name: f2_stats[f2_stats.Stat==name]['Landed'][fight] for name in names}}
        return returned

In [5]:
df_fight_data.head()

,fight_name,fighter_0,fighter_1,winner,method,round_end,date,fight_id
0,Tulio Palhares v Adriano Santos 1998-10-16,1899,1900,0,KO/TKO,1,1998-10-16,0
1,Ebenezer Fontes Braga v Jeremy Horn 1998-10-16,1898,1601,0,Submission,1,1998-10-16,1
2,Tsuyoshi Kohsaka v Pete Williams 1998-10-16,1813,1819,0,Decision,2,1998-10-16,2
3,Pat Miletich v Mikey Burnett 1998-10-16,1818,1893,0,Decision,3,1998-10-16,3
4,Pedro Rizzo v David Abbott 1998-10-16,1785,1784,0,KO/TKO,1,1998-10-16,4
